In [ ]:
import openslide as ops 
from PIL import Image
import os
import numpy as np
from glob import glob
def create_directory(path):
    if not os.path.exists(path):
        os.makedirs(path)
    return path

In [ ]:
slide_list=glob('../../data/BR_her2/*.ndpi')
create_directory('../../data/BR_her2_thumbnail/')
for slide_path in slide_list:
    slide=ops.OpenSlide(slide_path)
    thumb=slide.get_thumbnail((slide.level_dimensions[0][0]//32,slide.level_dimensions[0][1]//32))
    save_path=os.path.join('../../data/BR_her2_thumbnail/',os.path.basename(slide_path).replace('CODIPAI-','').replace('-SS','').replace('.ndpi','.png'))
    thumb.save(save_path)